# Accessing NCBI Databases with BioPython
(Víctor Sojo | vsojo@amnh.org)

Here we will be introducing **BioPython** and using it to interface with NCBI databases.

**References:** To prepare this document, I made very heavy use of:
+ The [_BioPython tutorial_](http://biopython.org/DIST/docs/tutorial/Tutorial.html).
+ Tiago Antao's book [_Bioinformatics with Python Cookbook_](https://www.packtpub.com/product/bioinformatics-with-python-cookbook-second-edition/9781789344691), which I highly recommend. You may be able to access it for free from your library (e.g., Columbia University has it in digital).

## Contents
&emsp;[Installing BioPython \(if you don't have it\)](#Installing-BioPython-\(if-you-don't-have-it\))<br/>
&emsp;[Importing necessary BioPython modules](#Importing-necessary-BioPython-modules)<br/>
&emsp;[Accessing NCBI via Entrez](#Accessing-NCBI-via-Entrez)<br/>
&emsp;&emsp;[Getting the list of available databases at NCBI](#Getting-the-list-of-available-databases-at-NCBI)<br/>
&emsp;[Download nucleotide records for a specific gene, by name, in a specific species](#Download-nucleotide-records-for-a-specific-gene,-by-name,-in-a-specific-species)<br/>
&emsp;[Downloading gene sequences given a list of IDs](#Downloading-gene-sequences-given-a-list-of-IDs)<br/>
&emsp;&emsp;[⚠️ Iterators are emptied after completing a single run ⚠️](#⚠️-Iterators-are-emptied-after-completing-a-single-run-⚠️)<br/>
&emsp;&emsp;[To keep data from an iterator permanently, convert it to a list \(inefficient\)](#To-keep-data-from-an-iterator-permanently,-convert-it-to-a-list-\(inefficient\))<br/>
&emsp;[Saving sequence records to files with Bio.SeqIO.write\(\)](#Saving-sequence-records-to-files-with-Bio.SeqIO.write\(\))<br/>
&emsp;[Accessing the NCBI Taxonomy database](#Accessing-the-NCBI-Taxonomy-database)<br/>
&emsp;&emsp;[Searching broadly using Entrez.esearch\(\)](#Searching-broadly-using-Entrez.esearch\(\))<br/>
&emsp;&emsp;[Fetching specific taxonomic records using IDs in Entrez.efetch\(\)](#Fetching-specific-taxonomic-records-using-IDs-in-Entrez.efetch\(\))<br/>

⚠️ I'm assuming you followed the Py201 notebook, in which we installed `conda`, set up an environment called `bioinfo`, and considered differences for Windows users ⚠️

First, let's make sure that you're using the appropriate environment:

Looking good for me. If you don't see `bioinfo` but instead `base` or `root`, you probably should close this notebook, stop jupyter by hitting **Quit** at the top-right of the main Jupyter browser webpage (or by issuing `jupyter notebook stop` on a new terminal tab or hitting Ctrl+C twice in the Terminal tab running Jupyter), then load your `conda` environment:
```bash
conda activate bioinfo
```
and restart Jupyter:
```bash
jupyter notebook
```

⚠️**Windows users:** Every time I write `!something` you should instead write **`!wsl`**`something`. For this to work, you need to have the Windows Subsystem for Linux (WSL) activated. You need to do that otherwise a significant part of the code in this workshop won't work for you. [Learn about the WSL here](https://docs.microsoft.com/en-us/windows/wsl/install-win10).

## Installing BioPython (if you don't have it)
You should have installed BioPython previously. If you haven't, you can do it inside this Jupyter Notebook itself:

In my case, I already have it, so `conda` just searches for updates and then lets me know that I do have it. It will probably take a couple of minutes if you don't have it installed. 

## Importing necessary BioPython modules

BioPython is enormous, and we don't need all of it here. For this lesson, we only need the following modules:

Module      | Use
:-----------|:-----------------------------------------
**Entrez**  | Programming interface to retrieve data from NCBI
**SeqIO**   | To read and write bio-sequences

## Accessing NCBI via Entrez
**Entrez** is NCBI's communication portal, so we will be using it here to download information from there.

⚠️ **Important** ⚠️ Always inform NCBI of your email address!

### Getting the list of available databases at NCBI
NCBI hosts a gigantic repository with multiple databases. Let's start by downloading the list of those databases:

You will probably recognise some of those names from your previous work or studies (e.g., `pubmed`, `nucleotide`, `protein`, `taxonomy`), but most of the others are probably unfamiliar.

Let's look for a gene in the `nucleotide` database next.

## Download `nucleotide` records for a specific gene, by name, in a specific species
We will download the `nucleotide` record _**references**_ that exactly match the `[Gene Name]` `"12S rRNA"` (this is the RNA of the small-subunit of the mitochondrial ribosome). We will do so specifying the red panda (_Ailurus fulgens_) as the `[Organism]` of interest here... because red pandas are great.

Let's take a look at that search result stored in the `handle` variable:

... well, that's not particularly helpful. This `ref_handle` variable holds the raw result of the communication to the NCBI database, including any actual results returned. To make use of these results (if any), we need to "parse" the handle into an actual record – as we did above when we got the list of databases. Once again we use the `.read()` method for this:

Let's take a proper look at the results variable:

The `ref_recs` variable seems to be a `dict`. To be honest, I can never make sense of dictionaries when they are printed like this. Let's use the `dict.items()` method and the marvelous f-strings to print it more legibly:

OK, that's much clearer. This `dict` contains the `Count` of records downloaded and the list of gene IDs, `IdList` , amongst other information.

An interesting one is `RetMax`, which here is the same as `Count` (`4`). The standard search is limited to 20 results by default. In our case, we only found 4 anyway, and this is why `RetMax` is `4`. But if you found and want to retrieve more than 20 results, you can specify e.g. `RetMax=150` as an additional parameter to `Entrez.esearch()` to get `150` records. To retrieve _all_ records, you could do `RetMax=recs['Count']`, which will get everything. However, be careful with that, since you may end up downloading a lot of data further down the line. At this point we're only downloading record IDs, so it's fine to download any number, but in the following section we will download actual nucleotide records (which can be very heavy).

In any case, we can use this dictionary to provide some useful output for the users (ourselves), for example:

Nice. This is useful information, but we only have the record IDs, not the actual gene sequences and info. Let's get that next.

## Downloading gene sequences given a list of IDs
So far, we have only downloaded the record _references_, not the gene records themselves. But we have the gene IDs, so we can go ahead and download the full gene records using those IDs.

We only have 4 results here, so this should be a trivial download. However, if you have a lot of results (because you changed `RetMax` above), you'll need to be careful about what you download and how. You may need to split in batches of a hundred or so.

Whatever you do, though, **don't download the records one by one** (e.g. with a `for` loop); it's inefficient and very bad Internet citizenship. Also, NCBI doesn't like it and may block your IP address (a good reason to give them your email address, just in case they want to talk to you to check you're not a bot).

We have just 4, so let's give the full list to `Entrez`:

Note that in the previous section we used `Entrez.esearch()` because we weren't sure exactly what we wanted. Here we're using `Entrez.efetch()` instead, because we know exactly which records we want.

Also, we specified the `gb` (GenBank) format, which has plenty of information. We could have used `fasta` instead, but `gb` is very good for exploring.

Let's parse that handle and take a look at the records we retrieved:

This also doesn't look very helpful, but in this case it actually is. As you can see, it's an `iterator` (read [here](https://www.w3schools.com/python/python_iterators.asp) if you want to know more), so we can go over it with a `for` loop:

---
Take a look at those results to make sure they make sense. Easily done because it's only `4`, but if they were thousands, you may need to parse through the names in some automated way (e.g. looking for desired terms such as `"12S"` and `"ribosomal"` in the `Description` and discarding any records that don't contain them).

### ⚠️ Iterators are emptied after completing a single run ⚠️
Let's try to run that last `for` loop again:

This time we get no output. This is because we had an `iterator`, which goes over each of its items only once, getting rid of the item as it produces it. This is **extremely efficient**, which is why Python insists that you use them whenever you can. However, it has limitations too, chiefly that you have no way of recovering the data once you've read it. In our case, this means we're forced to go back to NCBI again:

Good, we have our fresh iterator again. But every time we read something from it, each item we read is lost forever, until the iterator is empty. So, what do we do if we don't want to lose that data just by looking at it? One solution – not necessarily the best solution – is to convert the `iterator` to a `list`. 

### To keep data from an iterator permanently, convert it to a `list` (inefficient)
If we don't want to keep the data in our iterator more permanently in memory, we need to convert it to a `list`, which is an `iterable`. The slight difference in name means you can go over lists as many times as you want.

Now we can run the following `for` loop as many times as we wish:

---
Go ahead and re-run that last `for` loop. You'll see that this time the data is still there.

⚠️ **Warning** ⚠️ This code above seems great, but it can be a very bad idea if you have a lot of data. `list`s are a lot less efficient than `iterator`s, because the entire dataset needs to be loaded into memory with a list, as opposed to one item at a time with an iterator.

Here we have just `4` items, so that's not a problem, but if you have a lot of data, you'll want to keep it as an iterator, even if that means you can only look at it once. Just do look at it only once, do what you need to do with/to it, and move on to the next item. That's the properly _pythonic_ thing to do, more so with huge bioinformatics data.

The advantage of a list is of course that we can reuse the data as many times as we wish without having to go back to NCBI. Here, since we're just learning, and since it's only 4 items, turning an iterator to a `list` is acceptable. You'll find yourself turning iterators to lists many times, particularly with light data, and also with portions of your data as you're developing your code (since you need to explore the data and typically don't know what you'll find upon the first read).

But there's an even better way that lets us keep the data permanently: exporting it to a file. We explore that next.

## Saving sequence records to files with `Bio.SeqIO.write()`
If you wish to keep data such as a sequence record permanently, you can export it to a file, in some desired format. You can then load this data back in as needed. Here we will export our records to GenBank format.

First, let's use the shell to create a folder to hold the `GenBank` files:

(In my case, I already had that directory, so bash gives me a minor complaint)

We still have the records in the `gene_recs` list, so we could export them easily from there. However, in a real analysis, the optimal way to do this would be to create the files directly as you read them from the NCBI record. So, with apologies to NCBI and its worldwide users, we will go back there a third time.

We have our iterator again. But this time we will neither waste it by printing, nor turn it inefficiently into a list. Instead, we will do the proper thing and save each of the records to its own file.

Importantly, we would benefit from some automated way to access these files later, so let's store them into a list for now.

Here, we exported each of the records verbatim to a file, and we used the corresponding record's ID (`rec.id`) to name its file.

(Note that the record IDs don't match the gene IDs that we had from before. NCBI uses multiple identifiers for sequences. Right now it doesn't matter much to us, but for real bioinformatics work, you'll need to be very careful with which identifier you are using)

Let's take a quick look at one of those files:

Perfect! I suggest you also open this GenBank file externally to explore it properly (for example, you could use the JupyterLab browser).

#### Create an index file with the name and location of each of the GenBank, to be accessed later
We will create a file containing the list of names of all the GenBank files that we created above, so that we can find them easily in our later analyses:

Let's take a look:

We will use this list of files in the following lesson, when we will be looking into reading into those files to find the actual DNA sequences of the 12S rRNA gene.

## Accessing the NCBI Taxonomy database
Let's download the taxonomic information for the red panda.

### _Searching_ broadly using `Entrez.esearch()`
If all you have is the name of a species, this can be very ambiguous. For example, if you just look for "fox", how could NCBI know which fox you mean? But the red panda is unique (and so very pretty), so it should give a good result using **`Entrez.esearch()`**:

We have stored the results into a dictionary called `tax_records`. Let's print all of its keys and corresponding values:

Nice, we found a single result, and we now have its id in the `IdList`. We can use that id to download the full taxonomic record of the red panda (assuming that the number `9649` is a correct result).

### _Fetching_ specific taxonomic records using IDs in `Entrez.efetch()`
Now that we have the ID of the red panda, we can use it to download the full desired record.

This time, since we know exactly what we're looking for, we use `efetch`, instead of `esearch` which we used above when we were looking more broadly.

Let's also add the ID of the giant panda, which I know to be `9646`.


(remember that `Entrez` typically returns iterators, which you can only loop over once. Turn to list while you develop your code so that you don't have to go back to NCBI again and again, then remove the conversion — line 3 above – to leave as iterator in your final code)

Nice! There's a lot of very useful information there. We could for example extract the `Lineage` for each of these organisms very easily, and store it in a file that we can use later.

#### Homework
There is a rumour that red pandas are not related to giant pandas – that giant pandas belong to the larger bear family whereas red pandas are in their own group near the raccoons and other musteloids. Print out the `'family'` of the two animals to see what we get.

**Advanced real-world kind of homework:** Create a tab-separated file named `panda_taxonomies.tsv` that contains the following columns:
1. The TaxId.
1. The `CommonName`, if present, otherwise the GenbankCommonName.
1. The `Lineage`.
1. The following ranks of the `LineageEx`, if present, otherwise empty, each in its own column:
   1. `kingdom`.
   1. `phylum`.
   1. `order`.
   1. `family`.
   1. `genus`.